In [1]:
import keyring

SERVICE = 'OpenAI'
NAME = 'HomeworkAPIKey'

apikey = keyring.get_password(SERVICE, NAME)

In [2]:
import os
import pickle

import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [3]:
# Initialise LLM with required params
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0.9, openai_api_key=apikey, max_tokens=500)

In [4]:
# Load data

# Doesn't work - replaced with below code:
'''
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)
'''
...

Ellipsis

In [5]:
# Load data:

loaders = WebBaseLoader([
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()
len(data)

2

In [6]:
# Split data to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)
len(docs)

34

In [7]:
docs[0]

Document(page_content='Wall Street rises as Tesla soars on AI optimism', metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html', 'title': 'Wall Street rises as Tesla soars on AI optimism', 'description': 'Tesla (TSLA.O) rallied 10% after Morgan Stanley upgraded the electric car maker to ', 'language': 'en'})

In [8]:
# Create embeddings for these chunks and save them to FAISS index
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=apikey)

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [9]:
# Storing vector index create in local

file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex_openai, f)

In [10]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [11]:
# Retrieve similar embeddings for a given question and call LLM to retrieve final answer:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
# chain

In [12]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

# Deprecated:
# chain({"question": query}, return_only_outputs=True)
chain.invoke({"question": query})

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "driver seat.The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.Tata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".",
      "question": "what is the price of Tiago iCNG?"
    },
    {
      "context": "Tata Motors on Friday launched th

{'question': 'what is the price of Tiago iCNG?',
 'answer': ' The price of the Tiago iCNG is between Rs 6.55 lakh and Rs 8.1 lakh. \n',
 'sources': 'https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html'}